In [31]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [32]:
import json
import os
import glob

with open('data/slots/time_slots.json', 'r') as f:
    slots_data = json.load(f)

with open('data/obrony/obrony.json', 'r') as f:
    defences_data = json.load(f)

with open('data/people/przewodniczacy.json', 'r') as f:
    chairmans = json.load(f)

with open('data/people/workers.json', 'r') as f:
    members = json.load(f)

In [33]:
import os

folder_path = 'data/examples'

# Get all file names in the folder
file_names = os.listdir(folder_path)

# Sort file names numerically
sorted_file_names = sorted(file_names, key=lambda x: int(x.split('.')[0]))

# Create a list of file paths
examples_list = [os.path.join(folder_path, file_name) for file_name in sorted_file_names]

# Print the file paths
print(examples_list)

['data/examples\\0.json', 'data/examples\\1.json', 'data/examples\\2.json', 'data/examples\\3.json', 'data/examples\\4.json', 'data/examples\\5.json', 'data/examples\\6.json', 'data/examples\\7.json', 'data/examples\\8.json', 'data/examples\\9.json', 'data/examples\\10.json', 'data/examples\\11.json', 'data/examples\\12.json', 'data/examples\\13.json', 'data/examples\\14.json', 'data/examples\\15.json', 'data/examples\\16.json', 'data/examples\\17.json', 'data/examples\\18.json', 'data/examples\\19.json', 'data/examples\\20.json', 'data/examples\\21.json', 'data/examples\\22.json', 'data/examples\\23.json', 'data/examples\\24.json', 'data/examples\\25.json', 'data/examples\\26.json', 'data/examples\\27.json', 'data/examples\\28.json', 'data/examples\\29.json', 'data/examples\\30.json', 'data/examples\\31.json', 'data/examples\\32.json', 'data/examples\\33.json', 'data/examples\\34.json', 'data/examples\\35.json', 'data/examples\\36.json', 'data/examples\\37.json', 'data/examples\\38.js

In [34]:
# examples_list = [examples_list[34]]
# examples_list

In [35]:
from Defence_slots import Defence, Slot

defences = [Defence(person_data["student"], person_data["promotor"], person_data["recenzent"]) for person_data in defences_data]

slots = [Slot(index, time) for index, time in slots_data.items()]

In [36]:
from Environment import Environment
from State_action import Action
from algorithm import Qlearning
from create_data_for_Q import Examples_dataset

q_list = []

for example in examples_list[:2]:
    dataset = Examples_dataset(example, chairmans)
    availability = dataset.create_availability()
    env = Environment(defences, slots, chairmans, members, availability)
    q_algo = Qlearning(env)
    q_list.append(q_algo)

In [37]:
count_list = []
min_count = 30
best_schedule = None
i = 0

for q_algo in q_list:
    filled_Q_table = q_algo.qlearning_algorithm()
    schedule, count = q_algo.create_schedule()
    if count < min_count:
        min_count = count
        best_schedule = schedule
        i_min = i
    count_list.append(count)
    i += 1

Episode finished.
Episode finished.
Episode finished.


In [52]:
for defence in best_schedule:
    print(defence[0])

Defence exam: student: Ernest Pietuszyński, promoter: dr hab. T.Gambin, prof. PW, reviewer: prof. dr hab. P.Rokita
Defence exam: student: Mariusz Rzehorzek, promoter: dr inż. P.Zawistowski, reviewer: dr inż. G.Blinowski
Defence exam: student: Krzysztof Arciszewski, promoter: dr J.Komorowski, reviewer: dr inż. M.Kozłowski
Defence exam: student: Danuta Amsko, promoter: dr inż. M.Chwesiuk, reviewer: dr hab. R.Nowak, prof. PW


In [38]:
import pandas as pd

df = pd.DataFrame({'index': range(len(count_list)), 'value': count_list})

df.to_json('count.json', orient='records', lines=True)

In [39]:
print(min(count_list))

26


In [40]:
print(count_list)

[26, 26]


In [41]:
i_min

0

In [42]:
def print_defence_action(schedule):
    state, action, reward = schedule
    print(state)
    print(action)
    print(reward)

In [43]:
# print_defence_action(best_schedule[17])

In [44]:
# import pandas as pd

# best_schedule_defence, best_schedule_action = best_schedule

# # Convert schedule to a list of dictionaries
# schedule_dicts = [action.__dict__ for action in best_schedule_action]

# # Convert the list of dictionaries to a DataFrame
# df = pd.DataFrame(schedule_dicts)

# # Save DataFrame to a JSON file
# df.to_json('schedule.json', orient='records', lines=True)
